In [ ]:
%reload_ext autoreload
%autoreload 2

In [1]:
from config.config import MODEL_PATH, MODEL_CLASSES, MODEL_TYPE, DO_LOWER_CASE, BATCH_SIZE, DEVICE
from utils.load_data import *
from utils.utils import *
import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import pandas as pd


Devices: cuda


In [2]:
tokenizer = MODEL_CLASSES[MODEL_TYPE][2].from_pretrained(MODEL_PATH, do_lower_case=DO_LOWER_CASE)
if (MODEL_TYPE == "xlm"):
        tokenizer = MODEL_CLASSES[MODEL_TYPE][2].from_pretrained(MODEL_PATH, do_lowercase_and_remove_accent=DO_LOWER_CASE)

In [3]:
import json
data = json.load(open("data/test.json",encoding='utf-8'))

In [36]:
examples = []
output_ids = []

for testcase in data:
    
    test_id = testcase["__id__"]
    text_a = testcase["question"]
    
    for paragraph in testcase["paragraphs"]:
        answer_id = paragraph["id"]
        text_b = paragraph["text"]
        output_ids.append({"test_id": test_id, "answer_id": answer_id, "question": text_a, "answer": text_b})
        
        example = InputExample(guid="", text_a=remove_nonlatin(text_a), text_b=remove_nonlatin(text_b), label=0)
        examples.append(example)
        

In [5]:
features = convert_examples_to_features(examples, tokenizer,
            cls_token_at_end=bool(MODEL_TYPE in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if MODEL_TYPE in ['xlnet'] else 1,
            pad_on_left=bool(MODEL_TYPE in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if MODEL_TYPE in ['xlnet'] else 0)

# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

2019-11-08 03:13:38,028 - load_data.py - INFO - Writing example 0 of 2678
2019-11-08 03:13:50,350 - load_data.py - INFO - *** Example ***
2019-11-08 03:13:50,352 - load_data.py - INFO - guid: 
2019-11-08 03:13:50,352 - load_data.py - INFO - tokens: </s> To à</w> nhà</w> nào</w> cao</w> nhất</w> Việt</w> Nam</w> </s> The</w> Land mark</w> 81</w> là</w> một</w> to à</w> nhà</w> ch ọc</w> trời</w> trong</w> tổ</w> hợp</w> dự</w> án</w> Vin homes</w> Tân</w> Cảng</w> ,</w> một</w> dự</w> án</w> có</w> tổng</w> mức</w> đầu</w> tư</w> 40.000</w> tỷ</w> đồng</w> ,</w> do</w> Công</w> ty</w> Cổ</w> phần</w> Đầu</w> tư</w> xây</w> dựng</w> Tân</w> Liên</w> Phát</w> thuộc</w> Ving ro up</w> làm</w> chủ</w> đầu</w> tư</w> .</w> To à</w> tháp</w> cao</w> 81</w> tầng</w> ,</w> hiện</w> tại</w> là</w> to à</w> nhà</w> cao</w> nhất</w> Việt</w> Nam</w> và</w> là</w> to à</w> nhà</w> cao</w> nhất</w> Đông</w> Nam</w> Á</w> từ</w> tháng</w> 3</w> năm</w> 2018</w> .</w> </s>
2019-11-08 03:13:50,353 - lo

2019-11-08 03:13:50,377 - load_data.py - INFO - label: 0 (id = 0)
2019-11-08 03:13:50,380 - load_data.py - INFO - *** Example ***
2019-11-08 03:13:50,380 - load_data.py - INFO - guid: 
2019-11-08 03:13:50,381 - load_data.py - INFO - tokens: </s> Messi</w> thuận</w> chân</w> nào</w> </s> Ngoài</w> ra</w> anh</w> còn</w> một</w> chân</w> sút</w> phạt</w> chính</w> và</w> hay</w> đảm</w> nhận</w> các</w> quả</w> phạt</w> đền</w> của</w> đội</w> .</w> Khi</w> đề</w> cập</w> đến</w> kĩ</w> năng</w> dắt</w> bóng</w> của</w> anh</w> ,</w> Maradona</w> đã</w> từng</w> nói</w> :</w> "</w> Trái</w> bóng</w> dính</w> chặt</w> vào</w> chân</w> anh</w> ấy</w> ;</w> Trong</w> sự</w> nghiệp</w> của</w> mình</w> tôi</w> đã</w> xem</w> nhiều</w> cầu</w> thủ</w> xuất</w> chúng</w> chơi</w> bóng</w> nhưng</w> không</w> ai</w> giữ</w> bóng</w> tốt</w> như</w> Messi</w> .</w> "</w> Maradona</w> còn</w> cho</w> biết</w> rằng</w> ông</w> tin</w> Messi</w> là</w> cầu</w> thủ</w> xuất</w> sắc</w> nhất</w> thế<

In [22]:
out_dir = "runs/xlm_4/"
config = MODEL_CLASSES[MODEL_TYPE][0].from_pretrained(out_dir, num_labels=2, finetuning_task="zalo")
model = MODEL_CLASSES[MODEL_TYPE][1].from_pretrained(out_dir, from_tf=False, config=config)

In [23]:
distill_sampler = SequentialSampler(dataset)
distill_dataloader = DataLoader(dataset, sampler=distill_sampler, batch_size=BATCH_SIZE)

In [24]:
model.to(DEVICE)
model.eval()
i = 2

out_sent = []
out_logit_0 = []
out_logit_1 = []

out_label = []
for batch in tqdm(distill_dataloader):
    batch = tuple(t.to(DEVICE) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if MODEL_TYPE in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}

        outputs = model(**inputs)
        _, logits = outputs[:2]
        out_label.extend([0 if x[0] > x[1] else 1 for x in logits])

100%|██████████| 335/335 [00:21<00:00, 15.82it/s]


In [25]:
for k, l in zip(output_ids, out_label):
    k[out_dir] = l
        

In [33]:
df.loc[:,'Row_Total'] = df.sum(numeric_only=True, axis=1)

In [37]:
df = pd.DataFrame(output_ids)
df.to_csv("data/submission_xlm_id.csv", index=False)